In [2]:
!cd paylesshealth/ && dolt sql -q "select * from hospitals where cdm_indirect_url is not null;" -r csv > all_nulls.csv

In [52]:
import pandas as pd
# df = pd.read_csv('paylesshealth/hospitals.csv')
df = pd.read_csv('./paylesshealth-1/all_nulls.csv')

df2 = pd.read_csv('finished_possibles.csv')
ccn_list = df2["ccn"].tolist()

df = df[~df['ccn'].isin(ccn_list)]
# df = df.head(100)

In [38]:
import aiohttp
import asyncio
import re
import cgi
from pathlib import Path
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import traceback as tb

In [39]:
def create_headers(url):
    headers= {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.5",
    "Accept-Encoding": "gzip, deflate",
    "Connection": "keep-alive",
    "Upgrade-Insecure-Requests": "1",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "none",
    "Sec-Fetch-User": "?1",
    "Cache-Control": "max-age=0",
    "Host": urlparse(url).netloc,
    "Referer": urlparse(url).netloc
    }
    return headers

In [53]:
async def checker(session, url):
    find_strings = ["price transparency", "standard charges", "pricing", "billing", "prices", "price listing", "charges", "transparency", "standard services"]
    url_string = ["price-transparency", "standard-charges", "pricing", "billing", "prices", "price-list", "price-listing", "charges", "transparency", "standard-services"]
    if pd.isna(url):
        return pd.NA, pd.NA
    
    # if "/../" in url or "|" in url: # aiohttp is kinda not smart
    #     return False

    headers = create_headers(url)
    
    try:
        async with session.get(url, allow_redirects=True, ssl=False, raise_for_status=True, timeout=100, headers=headers) as r:
            if r:
                r = await r.text()
                soup = BeautifulSoup(r, "html.parser")
                
                r = r.lower()
                
                # Check for common price transparency names
                if any(string in r for string in find_strings):
                    matching_str = [string for string in find_strings if string in r]

                    possible_indirects = []
                    for t_url in soup.find_all('a'):
                        t_url = t_url.get("href")
                        
                        if t_url:
                            if any(string in t_url for string in url_string):
                                if t_url not in possible_indirects:
                                    possible_indirects.append(t_url)

                    if not possible_indirects:
                        # possible_indirects = pd.NA
                        return pd.NA, pd.NA
                    else:
                        possible_indirects = " | ".join(possible_indirects)
                    
                    return possible_indirects, matching_str
                
                else:
                    return pd.NA, pd.NA
                    
    except aiohttp.ClientResponseError as e:
#         print("ERROR", e, url)
        return pd.NA, pd.NA

    except asyncio.TimeoutError:
        return pd.NA, pd.NA

    except aiohttp.ClientConnectionError:
        return pd.NA, pd.NA    
    
    except KeyboardInterrupt:
        import os; os.exit()

    # except UnicodeDecodeError:
    #     tb.print_exc()        
    except Exception as e:
        tb.print_exc()
        print(e)
        return pd.NA, pd.NA

In [45]:
from tqdm.asyncio import tqdm

async def main():
    async with aiohttp.ClientSession(raise_for_status = True) as session:
        df[["possible_indirect", "found_string"]] = await tqdm.gather(*(checker(session, url) for url in df["homepage"]), leave=True, position=0)
        return df

In [54]:
t = await main()

 86%|████████▋ | 1112/1286 [01:02<00:06, 27.86it/s]Traceback (most recent call last):
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_22208\624924292.py", line 15, in checker
    r = await r.text()
  File "c:\Users\adria\Miniconda3\envs\scraper\lib\site-packages\aiohttp\client_reqrep.py", line 1085, in text
    return self._body.decode(  # type: ignore[no-any-return,union-attr]
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x97 in position 79817: invalid start byte


'utf-8' codec can't decode byte 0x97 in position 79817: invalid start byte


100%|██████████| 1286/1286 [01:38<00:00, 13.02it/s]
c:\Users\adria\Miniconda3\envs\scraper\lib\site-packages\numpy\core\fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


In [ ]:
t = await main()

 86%|████████▋ | 1112/1286 [01:02<00:06, 27.86it/s]Traceback (most recent call last):
  File "C:\Users\adria\AppData\Local\Temp\ipykernel_22208\624924292.py", line 15, in checker
    r = await r.text()
  File "c:\Users\adria\Miniconda3\envs\scraper\lib\site-packages\aiohttp\client_reqrep.py", line 1085, in text
    return self._body.decode(  # type: ignore[no-any-return,union-attr]
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x97 in position 79817: invalid start byte


'utf-8' codec can't decode byte 0x97 in position 79817: invalid start byte


100%|██████████| 1286/1286 [01:38<00:00, 13.02it/s]
c:\Users\adria\Miniconda3\envs\scraper\lib\site-packages\numpy\core\fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


In [59]:
t.dropna(subset=["found_string"], inplace=True)

t = t[t["homepage"].str.contains("https://www.va.gov/")==False] # Remove VA hospitals

t

,ccn,name,state,city,homepage,cdm_indirect_url,cdm_url,possible_indirect,found_string
1,053304,CHILDREN'S HOSPITAL OF ORANGE COUNTY,CA,ORANGE,https://www.choc.org/,https://www.choc.org/patients-family/pricing-t...,NaN,/patients-family/pricing-transparency/ | /pati...,"[pricing, billing, charges, transparency]"
2,053306,CHILDREN'S HOSPITAL AT MISSION,CA,MISSION VIEJO,https://www.choc.org/,NaN,NaN,/patients-family/pricing-transparency/ | /pati...,"[pricing, billing, charges, transparency]"
5,054009,RESNICK NEUROPSYCHIATRIC HOSPITAL AT UCLA,CA,LOS ANGELES,https://www.uclahealth.org/,NaN,NaN,/patients-families/billing-insurance | /patien...,[billing]
57,084001,DELAWARE PSYCHIATRIC CENTER,DE,NEW CASTLE,https://www.dhss.delaware.gov/dhss/main/maps/h...,NaN,NaN,https://delaware.gov/topics/transparency,[transparency]
114,030074,SELLS HOSPITAL,AZ,SELLS,https://www.ihs.gov/tucson/healthcarefacilitie...,NaN,NaN,/billings,"[billing, charges]"
...,...,...,...,...,...,...,...,...,...
1418,493302,CHILDREN'S HOSP OF RICHMOND OF THE VCU HEALTH SYS,VA,RICHMOND,https://www.chrichmond.org/,NaN,NaN,/chrichmond/patient-and-family-resources/insur...,[billing]
1456,501322,FERRY COUNTY MEMORIAL HOSPITAL,WA,REPUBLIC,https://www.fcphd.org/,NaN,NaN,#charges,"[pricing, billing, charges]"
1458,501328,MID VALLEY HOSPITAL,WA,OMAK,https://www.mvhealth.org/,NaN,NaN,https://www.mvhealth.org/patients-visitors/bil...,"[pricing, billing]"
1459,501336,MASON GENERAL HOSPITAL & FAMILY OF CLINICS,WA,SHELTON,https://www.masongeneral.com/,NaN,NaN,https://www.masongeneral.com/patients/disclosu...,"[pricing, billing, charges, transparency]"


In [60]:
t.to_csv("possible_indirects.csv", index=False)